# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 27 2022 9:46AM,240491,20,EXACT-8199613,"Exact-Rx, Inc.",Released
1,Jun 27 2022 9:46AM,240489,15,30009299,"Alliance Pharma, Inc.",Released
2,Jun 27 2022 9:46AM,240489,15,30009300,"Alliance Pharma, Inc.",Released
3,Jun 27 2022 9:46AM,240489,15,30009301,"Alliance Pharma, Inc.",Released
4,Jun 27 2022 9:46AM,240489,15,30009302,"Alliance Pharma, Inc.",Released
5,Jun 27 2022 9:46AM,240489,15,30009303,"Alliance Pharma, Inc.",Released
6,Jun 27 2022 9:46AM,240489,15,30009304,"Alliance Pharma, Inc.",Released
7,Jun 27 2022 9:46AM,240489,15,30009305,"Alliance Pharma, Inc.",Released
8,Jun 27 2022 9:46AM,240489,15,30009306,"Alliance Pharma, Inc.",Released
9,Jun 27 2022 9:46AM,240489,15,30009307,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,240486,Released,9
25,240487,Released,41
26,240488,Released,4
27,240489,Released,19
28,240491,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240486,NaN,9.0
240487,NaN,41.0
240488,NaN,4.0
240489,NaN,19.0
240491,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240455,0.0,4.0
240459,14.0,34.0
240462,21.0,15.0
240463,19.0,28.0
240464,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240455,0,4
240459,14,34
240462,21,15
240463,19,28
240464,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240455,0,4
1,240459,14,34
2,240462,21,15
3,240463,19,28
4,240464,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240455,,4
1,240459,14,34
2,240462,21,15
3,240463,19,28
4,240464,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 27 2022 9:46AM,240491,20,"Exact-Rx, Inc."
1,Jun 27 2022 9:46AM,240489,15,"Alliance Pharma, Inc."
20,Jun 27 2022 9:42AM,240487,10,ISDIN Corporation
61,Jun 27 2022 9:40AM,240488,10,"Amcyte Pharma, Inc."
65,Jun 27 2022 9:39AM,240484,10,ISDIN Corporation
105,Jun 27 2022 9:39AM,240486,10,MedStone Pharma LLC
114,Jun 27 2022 9:37AM,240485,10,Emerginnova
115,Jun 27 2022 9:36AM,240483,10,Beach Patient Assistance
117,Jun 27 2022 9:36AM,240483,10,Beach Products Inc
134,Jun 27 2022 9:35AM,240481,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 27 2022 9:46AM,240491,20,"Exact-Rx, Inc.",,1
1,Jun 27 2022 9:46AM,240489,15,"Alliance Pharma, Inc.",,19
2,Jun 27 2022 9:42AM,240487,10,ISDIN Corporation,,41
3,Jun 27 2022 9:40AM,240488,10,"Amcyte Pharma, Inc.",,4
4,Jun 27 2022 9:39AM,240484,10,ISDIN Corporation,,40
5,Jun 27 2022 9:39AM,240486,10,MedStone Pharma LLC,,9
6,Jun 27 2022 9:37AM,240485,10,Emerginnova,,1
7,Jun 27 2022 9:36AM,240483,10,Beach Patient Assistance,,19
8,Jun 27 2022 9:36AM,240483,10,Beach Products Inc,,19
9,Jun 27 2022 9:35AM,240481,10,ISDIN Corporation,,34


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 9:46AM,240491,20,"Exact-Rx, Inc.",1,
1,Jun 27 2022 9:46AM,240489,15,"Alliance Pharma, Inc.",19,
2,Jun 27 2022 9:42AM,240487,10,ISDIN Corporation,41,
3,Jun 27 2022 9:40AM,240488,10,"Amcyte Pharma, Inc.",4,
4,Jun 27 2022 9:39AM,240484,10,ISDIN Corporation,40,
5,Jun 27 2022 9:39AM,240486,10,MedStone Pharma LLC,9,
6,Jun 27 2022 9:37AM,240485,10,Emerginnova,1,
7,Jun 27 2022 9:36AM,240483,10,Beach Patient Assistance,19,
8,Jun 27 2022 9:36AM,240483,10,Beach Products Inc,19,
9,Jun 27 2022 9:35AM,240481,10,ISDIN Corporation,34,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 9:46AM,240491,20,"Exact-Rx, Inc.",1,
1,Jun 27 2022 9:46AM,240489,15,"Alliance Pharma, Inc.",19,
2,Jun 27 2022 9:42AM,240487,10,ISDIN Corporation,41,
3,Jun 27 2022 9:40AM,240488,10,"Amcyte Pharma, Inc.",4,
4,Jun 27 2022 9:39AM,240484,10,ISDIN Corporation,40,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 9:46AM,240491,20,"Exact-Rx, Inc.",1,NaN
1,Jun 27 2022 9:46AM,240489,15,"Alliance Pharma, Inc.",19,NaN
2,Jun 27 2022 9:42AM,240487,10,ISDIN Corporation,41,NaN
3,Jun 27 2022 9:40AM,240488,10,"Amcyte Pharma, Inc.",4,NaN
4,Jun 27 2022 9:39AM,240484,10,ISDIN Corporation,40,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 9:46AM,240491,20,"Exact-Rx, Inc.",1,0.0
1,Jun 27 2022 9:46AM,240489,15,"Alliance Pharma, Inc.",19,0.0
2,Jun 27 2022 9:42AM,240487,10,ISDIN Corporation,41,0.0
3,Jun 27 2022 9:40AM,240488,10,"Amcyte Pharma, Inc.",4,0.0
4,Jun 27 2022 9:39AM,240484,10,ISDIN Corporation,40,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4328570,541,35.0
12,480927,29,19.0
15,240489,19,0.0
16,480948,2,0.0
19,240455,4,0.0
20,721446,20,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4328570,541,35.0
1,12,480927,29,19.0
2,15,240489,19,0.0
3,16,480948,2,0.0
4,19,240455,4,0.0
5,20,721446,20,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,541,35.0
1,12,29,19.0
2,15,19,0.0
3,16,2,0.0
4,19,4,0.0
5,20,20,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,541.0
1,12,Released,29.0
2,15,Released,19.0
3,16,Released,2.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Executing,35.0,19.0,0.0,0.0,0.0,0.0
Released,541.0,29.0,19.0,2.0,4.0,20.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Executing,35.0,19.0,0.0,0.0,0.0,0.0
1,Released,541.0,29.0,19.0,2.0,4.0,20.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Executing,35.0,19.0,0.0,0.0,0.0,0.0
1,Released,541.0,29.0,19.0,2.0,4.0,20.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()